## Use case #1: using file inputParams.json (recommended use)

Execute the following cells only the first time running this notebook:

In [1]:
import os
os.chdir("..")

Loading the input parameters file in a new instance of the `controller`:

In [2]:
from synthDataGen.controller import DataControllerESIOS

controller = DataControllerESIOS("inputParams.json", "./synthDataGen/settings/")

Getting the first DataFrame. Then adjust it with the anual adjustments dictionary and resample it to the desired resolution:

In [3]:
df = controller.getDataFromSource()
df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
datetime_utc,,,,,,,,,,,,,,,,
2023-07-13 14:00:00,47.13,70.76,49.02,57.83,66.57,59.35,68.88,48.03,84.82,51.22,61.17,72.52,52.18,41.18,96.65,250.76
2023-07-13 15:00:00,46.69,69.39,49.22,57.82,67.03,59.94,67.73,46.13,84.61,51.62,60.91,72.56,50.76,44.16,101.95,248.97
2023-07-13 16:00:00,49.40,69.75,48.26,57.39,68.28,60.36,62.17,46.70,84.19,49.88,60.14,71.52,50.97,42.72,105.06,234.28
2023-07-13 17:00:00,48.49,72.14,46.20,54.63,64.98,59.13,64.60,49.28,78.77,48.76,58.40,70.10,52.16,41.41,106.94,258.69
2023-07-13 18:00:00,42.68,71.08,46.29,53.00,67.07,62.54,67.36,52.36,76.95,49.34,59.20,70.86,55.22,43.77,110.11,291.11
2023-07-13 19:00:00,40.51,77.65,40.48,51.58,66.37,63.63,69.52,61.95,78.64,48.54,61.25,71.52,57.20,43.93,109.12,300.53
2023-07-13 20:00:00,41.26,88.11,41.22,50.92,67.05,68.90,71.20,68.65,79.16,47.43,60.46,71.28,56.57,44.04,103.50,298.49
2023-07-13 21:00:00,36.94,81.67,39.70,46.56,63.89,61.27,72.42,62.87,73.22,43.93,58.45,70.12,53.81,39.30,98.42,278.39
2023-07-13 22:00:00,40.34,74.55,36.91,45.67,59.02,66.38,68.68,63.23,68.01,38.49,57.78,67.60,56.50,36.34,104.12,290.36


In [4]:
df = controller.performAnualAdjustments(df)
df

[2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
datetime_utc,,,,,,,,,,,,,,,,
2023-07-13 14:00:00,471.3,707.6,490.2,578.3,665.7,593.5,688.8,480.3,848.2,512.2,611.7,725.2,521.8,411.8,966.5,2507.6
2023-07-13 15:00:00,466.9,693.9,492.2,578.2,670.3,599.4,677.3,461.3,846.1,516.2,609.1,725.6,507.6,441.6,1019.5,2489.7
2023-07-13 16:00:00,494.0,697.5,482.6,573.9,682.8,603.6,621.7,467.0,841.9,498.8,601.4,715.2,509.7,427.2,1050.6,2342.8
2023-07-13 17:00:00,484.9,721.4,462.0,546.3,649.8,591.3,646.0,492.8,787.7,487.6,584.0,701.0,521.6,414.1,1069.4,2586.9
2023-07-13 18:00:00,426.8,710.8,462.9,530.0,670.7,625.4,673.6,523.6,769.5,493.4,592.0,708.6,552.2,437.7,1101.1,2911.1
2023-07-13 19:00:00,405.1,776.5,404.8,515.8,663.7,636.3,695.2,619.5,786.4,485.4,612.5,715.2,572.0,439.3,1091.2,3005.3
2023-07-13 20:00:00,412.6,881.1,412.2,509.2,670.5,689.0,712.0,686.5,791.6,474.3,604.6,712.8,565.7,440.4,1035.0,2984.9
2023-07-13 21:00:00,369.4,816.7,397.0,465.6,638.9,612.7,724.2,628.7,732.2,439.3,584.5,701.2,538.1,393.0,984.2,2783.9
2023-07-13 22:00:00,403.4,745.5,369.1,456.7,590.2,663.8,686.8,632.3,680.1,384.9,577.8,676.0,565.0,363.4,1041.2,2903.6


In [5]:
df = controller.resampleOnAxis0(df)

import pandas as pd
pd.set_option('display.precision', 2)

df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
2023-07-13 14:00:00,471.30,707.60,490.20,578.30,665.70,593.50,688.80,480.30,848.20,512.20,611.70,725.20,521.80,411.80,966.50,2507.60
2023-07-13 14:15:00,466.23,702.80,491.66,578.26,664.93,594.68,692.26,473.24,846.87,515.60,611.35,726.34,516.58,424.00,981.86,2524.28
2023-07-13 14:30:00,463.81,698.92,492.48,578.23,665.44,596.06,691.50,467.72,846.07,517.41,610.81,726.79,512.47,433.03,995.82,2526.86
2023-07-13 14:45:00,464.03,695.95,492.66,578.21,667.23,597.63,686.51,463.74,845.82,517.60,610.05,726.54,509.48,438.90,1008.36,2515.33
2023-07-13 15:00:00,466.90,693.90,492.20,578.20,670.30,599.40,677.30,461.30,846.10,516.20,609.10,725.60,507.60,441.60,1019.50,2489.70
2023-07-13 15:15:00,472.41,692.76,491.10,578.20,674.64,601.36,663.87,460.41,846.92,513.19,607.94,723.96,506.83,441.14,1029.23,2449.96
2023-07-13 15:30:00,480.57,692.53,489.37,578.21,680.27,603.52,646.21,461.05,848.28,508.58,606.58,721.62,507.18,437.51,1037.55,2396.12
2023-07-13 15:45:00,488.67,693.89,486.65,577.14,684.03,604.67,630.19,463.25,847.50,503.43,604.55,718.67,508.26,432.33,1044.62,2352.34
2023-07-13 16:00:00,494.00,697.50,482.60,573.90,682.80,603.60,621.70,467.00,841.90,498.80,601.40,715.20,509.70,427.20,1050.60,2342.80
2023-07-13 16:15:00,496.57,703.37,477.22,568.49,676.57,600.31,620.72,472.30,831.47,494.69,597.12,711.20,511.49,422.13,1055.49,2367.49


# Use case #2: passing parameters


In [8]:
from datetime import datetime

d = datetime(2023, 6, 5, 7, 0)

df = controller.getDataFromSource(initialYear=2018, initDatetime=d, hoursAhead=6)
df

,2018,2019,2020,2021,2022
datetime_utc,,,,,
2023-06-05 06:00:00,67.83,53.13,37.07,81.00,193.53
2023-06-05 07:00:00,69.11,50.74,37.06,81.43,177.57
2023-06-05 08:00:00,69.87,50.44,35.33,85.35,174.79
2023-06-05 09:00:00,69.09,50.50,34.96,85.31,177.76
2023-06-05 10:00:00,68.72,51.26,35.24,85.29,177.64
2023-06-05 11:00:00,69.03,49.11,35.13,85.36,176.71
2023-06-05 12:00:00,69.02,47.22,32.09,83.26,173.96


In [9]:
df = controller.performAnualAdjustments(df, adjustmentsDict={2018: 1.012, 2019: 1.023, 2020: 1.0145, 2021: 1.03, 2022: 1.08})
df

[2018, 2019, 2020, 2021, 2022]


,2018,2019,2020,2021,2022
datetime_utc,,,,,
2023-06-05 06:00:00,68.64,54.35,37.61,83.43,209.01
2023-06-05 07:00:00,69.94,51.91,37.60,83.87,191.78
2023-06-05 08:00:00,70.71,51.60,35.84,87.91,188.77
2023-06-05 09:00:00,69.92,51.66,35.47,87.87,191.98
2023-06-05 10:00:00,69.54,52.44,35.75,87.85,191.85
2023-06-05 11:00:00,69.86,50.24,35.64,87.92,190.85
2023-06-05 12:00:00,69.85,48.31,32.56,85.76,187.88


In [10]:
df = controller.resampleOnAxis0(df, sampleFreqInMins=20, method="spline", order=3)

import pandas as pd
pd.set_option('display.precision', 2)

df

,2018,2019,2020,2021,2022
2023-06-05 06:00:00,68.64,54.35,37.61,83.43,209.01
2023-06-05 06:20:00,69.25,53.31,37.41,83.74,201.75
2023-06-05 06:40:00,69.70,52.60,36.97,84.39,196.71
2023-06-05 07:00:00,69.94,51.91,37.60,83.87,191.78
2023-06-05 07:20:00,70.24,51.79,36.42,85.63,190.74
2023-06-05 07:40:00,70.36,51.62,36.27,86.21,189.39
2023-06-05 08:00:00,70.71,51.60,35.84,87.91,188.77
2023-06-05 08:20:00,70.36,51.59,36.12,87.20,188.97
2023-06-05 08:40:00,70.28,51.67,36.09,87.61,189.48
2023-06-05 09:00:00,69.92,51.66,35.47,87.87,191.98
